In [1]:
import os
from typing import Optional, TypedDict, Annotated
from langchain_core.documents.base import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import PyPDFLoader
from langchain_unstructured import UnstructuredLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.vectorstores.base import VectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain_experimental.utilities import PythonREPL
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import tool
import warnings
import dotenv

warnings.filterwarnings("ignore", )

In [2]:
# .env파일 읽기
dotenv.load_dotenv()

# os라이브러리 이용한 읽기
# os.environ['OPENAI_API_KEY'] = "API-Key"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [3]:
# LLM Load

llm = ChatOpenAI(model="gpt-4o-mini", 
                 temperature=0.)

In [4]:
llm.invoke("1+1은?")

AIMessage(content='1 + 1은 2입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-61c0fdf6-981c-40d4-bcf6-955611621e7a-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
prompt = ChatPromptTemplate.from_messages([
    # MessagesPlaceholder(variable_name="context"),
    ("system", 
    """

    context : {context}
    
    당신은 언제나 고객에게 최선을 다해 답변을 하며 말투는 굉장히 친근합니다. 직업은 전문 상담원입니다. 답변 시, 아래의 규칙을 지켜야만 합니다.
    규칙 1. 주어진 context만을 이용하여 답변해야합니다. 
    규칙 2. 주어진 context에서 답변을 할 수 없다면 "해당 문의는 010-2255-3366으로 연락주시면 도와드리겠습니다. 영업 시간은 오전 10시-오후 6시입니다." 라고 대답하세요.
    규칙 3. 문자열에 A1, A2, A11, A22 등 필요 없는 문자는 제거한 뒤 출력합니다.
    규칙 4. 항상 친절한 말투로 응대합니다.
    규칙 5. 웹사이트 링크를 그대로 출력합니다. 대소문자를 명확하게 구분하세요.
    """),
    ("human", "{query}")
])

In [6]:
# Document Embedding

def embedding_file(file="Practice_document.pdf"):
    print("Embedding 시작")
    Embeddings = OpenAIEmbeddings()
    print("Embedding 완료!\n파일 로딩중")
    loader = UnstructuredLoader(file_path=f"./files/{file}")
    print("파일 로딩 완료!\n텍스트 스플리터 생성")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    split_docs = loader.load_and_split(text_splitter)
    print("문서 분할 완료")
    vector_store = FAISS.from_documents(split_docs, Embeddings)
    print("벡터 저장소 생성 완료")

    retriever = vector_store.as_retriever()
    print("벡터 리트리버 생성 완료")

    return retriever

In [7]:
retriever = embedding_file()

Embedding 시작
Embedding 완료!
파일 로딩중
파일 로딩 완료!
텍스트 스플리터 생성


INFO: pikepdf C++ to Python logger bridge initialized


문서 분할 완료


INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: Loading faiss with AVX2 support.
INFO: Successfully loaded faiss with AVX2 support.
INFO: Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes.


벡터 저장소 생성 완료
벡터 리트리버 생성 완료


In [8]:
docs = retriever.invoke("개강하는 날짜가 언제야?")

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [9]:
docs

[Document(id='d3bdee7e-08ff-4bfd-8a98-3275f53314c2', metadata={'source': './files/Practice_document.pdf', 'coordinates': {'points': ((71.999997, 456.5999809749999), (71.999997, 471.4945116343945), (118.70339260602509, 471.4945116343945), (118.70339260602509, 456.5999809749999)), 'system': 'PixelSpace', 'layout_width': 842.88, 'layout_height': 1191.12}, 'file_directory': './files', 'filename': 'Practice_document.pdf', 'languages': ['eng'], 'last_modified': '2025-03-10T18:14:18', 'page_number': 1, 'filetype': 'application/pdf', 'category': 'Title', 'element_id': '9b95bc223f875c62c26b62f788238a61'}, page_content='모집 기간'),
 Document(id='0bcd6208-25a7-4d1f-b9e2-6cf5795b2273', metadata={'source': './files/Practice_document.pdf', 'coordinates': {'points': ((89.26874628046875, 478.19998007499987), (89.26874628046875, 490.11560460251553), (321.37798963324985, 490.11560460251553), (321.37798963324985, 478.19998007499987)), 'system': 'PixelSpace', 'layout_width': 842.88, 'layout_height': 1191.12}